# A Macroscopically Estimating Model with Endogeneity Considered for Lane-mean Speeds and Speed Deviations (3lanes)

## 1 Structuring Data

### 1.1 Import Data

In [116]:
import numpy as np
import pandas as pd
import xlrd
from linearmodels import IV3SLS
import datetime
import math
import os
from dateutil.relativedelta import * # for incement of month

In [117]:
# filelist = []
# file_dir = 'G:\\Modeling Endogeneity\\Data'

# for root, dirs, files in os.walk(file_dir):
#     filelist = files

# file_200m_2lanes = [file for file in filelist if '200m_2lanes' in file]

In [118]:
data_up_fs = pd.DataFrame()
data_do_fs = pd.DataFrame()
data_up_o = pd.DataFrame()
data_up_tftp = pd.DataFrame()
data_up_vmtvht = pd.DataFrame()

# 12 for the number of month
for num in range(12):
    monnum = num + 1
    # flow,speed
    file_up_fs = 'G:\\Modeling Endogeneity\\Data\\100m_3lanes_up_fs_' + str(monnum) + '.xlsx'
    file_do_fs = 'G:\\Modeling Endogeneity\\Data\\100m_3lanes_do_fs_' + str(monnum) + '.xlsx'
    # occupancy
    file_up_o = 'G:\\Modeling Endogeneity\\Data\\100m_3lanes_up_o_' + str(monnum) + '.xlsx'
    file_do_o = 'G:\\Modeling Endogeneity\\Data\\100m_3lanes_do_o_' + str(monnum) + '.xlsx'
    # truck flow, truck percentage
    file_up_tftp = 'G:\\Modeling Endogeneity\\Data\\100m_3lanes_up_tftp_' + str(monnum) + '.xlsx'
    # file_do_tftp = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_tftp_1.xlsx'
    # truck vmt, trauck vht
    file_up_vmtvht = 'G:\\Modeling Endogeneity\\Data\\100m_3lanes_up_vmtvht_' + str(monnum) + '.xlsx'
    # file_do_vmtvht = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_do_vmtvht_1.xlsx'

    data_up_fs_t = pd.read_excel(file_up_fs)
    data_do_fs_t = pd.read_excel(file_do_fs)
    data_up_o_t = pd.read_excel(file_up_o)
    # data_do_o = pd.read_excel(file_do_o)
    data_up_tftp_t = pd.read_excel(file_up_tftp)
    # data_do_tftp = pd.read_excel(file_do_tftp)
    data_up_vmtvht_t = pd.read_excel(file_up_vmtvht)
    # data_do_vmtvht = pd.read_excel(file_do_vmtvht)
    data_up_fs = data_up_fs.append(data_up_fs_t, sort=True)
    data_do_fs = data_do_fs.append(data_do_fs_t, sort=True)
    data_up_o = data_up_o.append(data_up_o_t, sort=True)
    data_up_tftp = data_up_tftp.append(data_up_tftp_t, sort=True)
    data_up_vmtvht = data_up_vmtvht.append(data_up_vmtvht_t, sort=True)

In [119]:
data_up_vmtvht.head() # this command lane is used to observe the order of variables in the dataframe for changing the keyword easier.

,# Lane Points,% Observed,5 Minutes,Lane 1 Truck VHT (Veh-Hours),Lane 1 Truck VMT (Veh-Miles),Lane 2 Truck VHT (Veh-Hours),Lane 2 Truck VMT (Veh-Miles),Lane 3 Truck VHT (Veh-Hours),Lane 3 Truck VMT (Veh-Miles),Truck VHT (Veh-Hours),Truck VMT (Veh-Miles)
0,3,0,1/08/2017 0:00,0,0,0.0,0.0,0,0.0,0.0,0.0
1,3,0,1/08/2017 0:05,0,0,0.0,0.0,0,0.0,0.0,0.0
2,3,0,1/08/2017 0:10,0,0,0.0,0.0,0,0.0,0.0,0.0
3,3,0,1/08/2017 0:15,0,0,0.0,0.0,0,0.0,0.0,0.0
4,3,0,1/08/2017 0:20,0,0,0.0,0.0,0,0.0,0.0,0.0


In [120]:
data_do_fs.describe()

,# Lane Points,% Observed,Flow (Veh/5 Minutes),Lane 1 Flow (Veh/5 Minutes),Lane 1 Speed (mph),Lane 2 Flow (Veh/5 Minutes),Lane 2 Speed (mph),Lane 3 Flow (Veh/5 Minutes),Lane 3 Speed (mph),Speed (mph)
count,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000
mean,2.999876,99.081810,103.104723,39.265572,71.555331,21.900447,69.347225,41.938705,61.928228,66.898825
std,0.019294,9.538328,65.602294,27.209621,1.002506,15.829720,1.044097,24.350263,3.197016,1.616181
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,100.000000,37.000000,13.000000,71.400000,7.000000,69.100000,18.000000,61.100000,66.200000
50%,3.000000,100.000000,111.000000,40.000000,71.800000,22.000000,69.600000,47.000000,62.500000,67.000000
75%,3.000000,100.000000,157.000000,60.000000,71.900000,34.000000,69.800000,63.000000,62.900000,67.700000
max,3.000000,100.000000,306.000000,135.000000,75.000000,80.000000,73.700000,104.000000,72.700000,72.500000


In [121]:
# Caution: use dataframe.head() to observe the name and the position of every column
data_up = data_up_fs
data_up = data_up.iloc[:,2:11]
data_up.columns = ['t','f','f1','s1','f2','s2','f3','s3','s']
data_up[['o1','o2','o3']] = data_up_o.iloc[:,[3,4,5]]
data_up[['tf1','tp1','tf2','tp2','tf3','tp3']] = data_up_tftp.iloc[:,[3,4,5,6,7,8]]
data_up[['vht1','vmt1','vht2','vmt2','vht3','vmt3']] = data_up_vmtvht.iloc[:,[3,4,5,6,7,8]]
data_up[['s_do1','s_do2','s_do3']] = data_do_fs.iloc[:,[5,7,9]]
data_up = data_up.reset_index(drop=True)

In [122]:
data_up.describe()

,f,f1,s1,f2,s2,f3,s3,s,o1,o2,...,tp3,vht1,vmt1,vht2,vmt2,vht3,vmt3,s_do1,s_do2,s_do3
count,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,24178.000000,...,24178.000000,24178.0,24178.0,24178.000000,24178.000000,24178.0,24178.000000,24178.000000,24178.000000,24178.000000
mean,115.004715,29.025974,69.141873,45.566259,64.754628,40.412482,60.047150,64.092965,2.224196,3.563020,...,3.278696,0.0,0.0,0.000008,0.083468,0.0,0.246608,71.555331,69.347225,61.928228
std,49.757355,17.441708,1.344004,18.366204,2.001877,15.660536,1.378888,1.845376,0.951848,1.155127,...,3.459209,0.0,0.0,0.000909,0.189851,0.0,0.248999,1.002506,1.044097,3.197016
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,68.000000,12.000000,68.600000,28.000000,64.100000,27.000000,59.500000,63.300000,1.400000,2.600000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,71.400000,69.100000,61.100000
50%,122.000000,30.000000,69.200000,48.000000,64.600000,42.000000,60.000000,63.700000,2.300000,3.600000,...,3.000000,0.0,0.0,0.000000,0.000000,0.0,0.200000,71.800000,69.600000,62.500000
75%,156.000000,42.000000,69.500000,61.000000,65.000000,54.000000,60.500000,64.400000,3.000000,4.500000,...,5.800000,0.0,0.0,0.000000,0.200000,0.0,0.400000,71.900000,69.800000,62.900000
max,313.000000,126.000000,76.000000,135.000000,79.700000,95.000000,67.800000,75.000000,7.800000,12.800000,...,50.000000,0.0,0.0,0.100000,2.800000,0.0,1.300000,75.000000,73.700000,72.700000


###  1.2 Delete Outlier

In [123]:
data_up['t'] = pd.to_datetime(data_up['t'])
data = pd.DataFrame()
for month in range(12):
    
    for day in range(7): # There are 7 days data in each month
        # Select the data in the same day
        begin = data_up['t'][0] + datetime.timedelta(days=day) + relativedelta(months=month)
        end = data_up['t'][0] + datetime.timedelta(days=day+1) + relativedelta(months=month)
        interval = (data_up['t']>begin) & (data_up['t']<end)
        data_up_t = data_up[interval]

        # Computing Interquantile Range (IQR)
        upper_q_s1 = data_up_t['s1'].quantile(0.75)
        lower_q_s1 = data_up_t['s1'].quantile(0.25)
        IQR_s1 = upper_q_s1 - lower_q_s1
        upper_q_s2 = data_up_t['s2'].quantile(0.75)
        lower_q_s2 = data_up_t['s2'].quantile(0.25)
        IQR_s2 = upper_q_s2 - lower_q_s2
        upper_q_s3 = data_up_t['s3'].quantile(0.75)
        lower_q_s3 = data_up_t['s3'].quantile(0.25)
        IQR_s3 = upper_q_s3 - lower_q_s3
        
        # Delecting the records which are outlier in s1,s2 and s3.
        data_up_t = data_up_t[((lower_q_s1-1.5*IQR_s1) <= data_up_t['s1'])& (data_up_t['s1'] <= (upper_q_s1+1.5*IQR_s1))]
        data_up_t = data_up_t[((lower_q_s2-1.5*IQR_s2) <= data_up_t['s2'])& (data_up_t['s2'] <= (upper_q_s2+1.5*IQR_s2))]
        data_up_t = data_up_t[((lower_q_s3-1.5*IQR_s3) <= data_up_t['s3'])& (data_up_t['s3'] <= (upper_q_s3+1.5*IQR_s3))]

        data = data.append(data_up_t, sort = True)
        
data_up = data
data_up.describe()

,f,f1,f2,f3,o1,o2,o3,s,s1,s2,...,tf3,tp1,tp2,tp3,vht1,vht2,vht3,vmt1,vmt2,vmt3
count,22057.000000,22057.000000,22057.000000,22057.000000,22057.000000,22057.000000,22057.000000,22057.000000,22057.000000,22057.000000,...,22057.000000,22057.0,22057.000000,22057.000000,22057.0,22057.0,22057.0,22057.0,22057.000000,22057.000000
mean,111.981502,27.636623,44.435961,39.908918,2.150583,3.488090,3.516652,64.063350,69.154241,64.765467,...,1.607064,0.0,0.831618,3.092551,0.0,0.0,0.0,0.0,0.068364,0.235032
std,48.998414,16.523942,18.039125,15.565151,0.903724,1.116876,1.165828,1.621011,1.103789,1.655617,...,1.727535,0.0,2.743534,3.360899,0.0,0.0,0.0,0.0,0.146043,0.247179
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,56.800000,62.600000,56.700000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
25%,65.000000,12.000000,27.000000,26.000000,1.300000,2.500000,2.600000,63.300000,68.700000,64.200000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
50%,114.000000,28.000000,45.000000,40.000000,2.100000,3.500000,3.600000,63.800000,69.200000,64.600000,...,1.000000,0.0,0.000000,2.600000,0.0,0.0,0.0,0.0,0.000000,0.200000
75%,154.000000,41.000000,60.000000,53.000000,2.900000,4.400000,4.500000,64.300000,69.500000,64.900000,...,3.000000,0.0,1.400000,5.800000,0.0,0.0,0.0,0.0,0.200000,0.400000
max,294.000000,125.000000,127.000000,89.000000,7.300000,8.000000,6.500000,72.300000,73.600000,75.600000,...,8.000000,0.0,50.000000,50.000000,0.0,0.0,0.0,0.0,1.900000,1.200000


### 1.3 Construct Variables

In [124]:
# Delecting the records whose f1 = 0, f2 = 0 or f3 = 0
data_up = data_up[(data_up['f1']!=0) & (data_up['f2']!=0) & (data_up['f3']!=0)]
data_up.describe()

,f,f1,f2,f3,o1,o2,o3,s,s1,s2,...,tf3,tp1,tp2,tp3,vht1,vht2,vht3,vmt1,vmt2,vmt3
count,21910.000000,21910.000000,21910.000000,21910.000000,21910.000000,21910.000000,21910.000000,21910.000000,21910.000000,21910.000000,...,21910.000000,21910.0,21910.000000,21910.000000,21910.0,21910.0,21910.0,21910.0,21910.000000,21910.000000
mean,112.700411,27.810452,44.714149,40.175810,2.164340,3.509836,3.540187,64.027462,69.133199,64.729005,...,1.617846,0.0,0.805586,3.113300,0.0,0.0,0.0,0.0,0.068462,0.236609
std,48.364408,16.440382,17.772784,15.271144,0.890855,1.088128,1.133644,1.563713,1.076686,1.598964,...,1.728281,0.0,2.567430,3.362564,0.0,0.0,0.0,0.0,0.146152,0.247253
min,4.000000,1.000000,1.000000,1.000000,0.000000,0.100000,0.000000,56.800000,62.600000,56.700000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
25%,65.000000,12.000000,28.000000,26.000000,1.300000,2.600000,2.700000,63.300000,68.700000,64.200000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
50%,115.000000,28.000000,46.000000,41.000000,2.200000,3.500000,3.600000,63.700000,69.200000,64.600000,...,1.000000,0.0,0.000000,2.600000,0.0,0.0,0.0,0.0,0.000000,0.200000
75%,154.000000,41.000000,60.000000,53.000000,2.900000,4.400000,4.500000,64.300000,69.500000,64.900000,...,3.000000,0.0,1.400000,5.800000,0.0,0.0,0.0,0.0,0.200000,0.400000
max,294.000000,125.000000,127.000000,89.000000,7.300000,8.000000,6.500000,72.300000,73.600000,75.600000,...,8.000000,0.0,50.000000,50.000000,0.0,0.0,0.0,0.0,1.900000,1.200000


In [125]:
# Spliting the data into three dataframes for lane1, lane2 and lane3
lane1list = ['t']+[col for col in data_up.columns if '1' in col]
data1 = data_up[lane1list]
lane2list = ['t']+[col for col in data_up.columns if '2' in col]
data2 = data_up[lane2list]
lane3list = ['t']+[col for col in data_up.columns if '3' in col]
data3 = data_up[lane3list]
data1.head()

,t,f1,o1,s1,s_do1,tf1,tp1,vht1,vmt1
1,2017-01-08 00:05:00,13,1.4,69.3,71.8,0,0,0,0
2,2017-01-08 00:10:00,12,1.4,69.0,71.8,0,0,0,0
3,2017-01-08 00:15:00,11,1.2,69.2,71.7,0,0,0,0
4,2017-01-08 00:20:00,12,1.5,68.7,71.5,0,0,0,0
5,2017-01-08 00:25:00,10,1.3,68.9,71.4,0,0,0,0


In [126]:
# Change the key of every dataframe for iteration later.
variables = ['t','f','o','s','s_do','tf','tp','vht','vmt']
data1.columns = variables
data2.columns = variables
data3.columns = variables
data1.loc[:,'s2'] = data2.loc[:,'s']
# Because lane2 is adjacent to lane1 and lane3 at the same time
data2.loc[:,'s1'] = data1.loc[:,'s']
data2.loc[:,'s3'] = data3.loc[:,'s']
data3.loc[:,'s2'] = data2.loc[:,'s']
data2.head()

E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
E:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,t,f,o,s,s_do,tf,tp,vht,vmt,s1,s3
1,2017-01-08 00:05:00,30,2.7,64.7,69.8,0,0.0,0.0,0.0,69.3,60.0
2,2017-01-08 00:10:00,29,2.7,64.4,69.6,0,0.0,0.0,0.0,69.0,59.7
3,2017-01-08 00:15:00,26,2.5,64.6,69.6,0,0.0,0.0,0.0,69.2,59.9
4,2017-01-08 00:20:00,29,2.7,64.1,69.6,0,0.0,0.0,0.0,68.7,59.4
5,2017-01-08 00:25:00,27,2.6,64.2,69.5,0,0.0,0.0,0.0,68.9,59.6


In [127]:
# use dummy variablesS
for lanedata in [data1, data2, data3]:
    # ln(speed)
    lanedata.loc[:,'lns'] = lanedata['s'].map(lambda x: math.log(x))
    # truck speed
    lanedata.loc[:,'ts'] = 999
    lanedata.loc[lanedata['vht']!=0, 'ts'] =  lanedata['vmt']/lanedata['vht']
#     lanedata['ts'] = lanedata.apply(lambda x: 999.0 if x['vht'] == 0 else x['vmt']/x['vht'], axis = 1) 
#     '''axis : {0 or ‘index’, 1 or ‘columns’}, default 0
#         Axis along which the function is applied:
#         0 or ‘index’: apply function to each column.
#         1 or ‘columns’: apply function to each row.'''
    # traffic flow indicator and high truck flow indicator
    lanedata.loc[:,'fi'] = 0
    lanedata.loc[:,'htfi'] = 0
    lanedata.loc[lanedata['f']<75, 'fi'] = 1
    lanedata.loc[lanedata['tf']>100, 'htfi'] = 1
    # relative truck percentage indicator
    lanedata.loc[:,'tpia'] = 0
    lanedata.loc[:,'tpib'] = 0
    lanedata.loc[(lanedata['tp']>0.6)&(lanedata['f']<50), 'tpia'] = 1
    lanedata.loc[(lanedata['tp']<=0.2)&(lanedata['f']>200), 'tpib'] = 1
    # seasonal indicator (fall will be reference)
    lanedata.loc[:,'seasona'] = 0
    lanedata.loc[:,'seasonb'] = 0
    lanedata.loc[:,'seasonc'] = 0
    lanedata.loc[(2<lanedata['t'].dt.month) & (lanedata['t'].dt.month<6), 'seasona'] = 1 # Spring
    lanedata.loc[(5<lanedata['t'].dt.month) & (lanedata['t'].dt.month<9), 'seasonb'] = 1
    lanedata.loc[(8<lanedata['t'].dt.month) & (lanedata['t'].dt.month<12), 'seasonc'] = 1
    # time-of-week indicator
    lanedata.loc[:,'weeka'] = 0
    lanedata.loc[:,'weekb'] = 0
    lanedata.loc[:,'weekc'] = 0
    lanedata.loc[:,'weekd'] = 0
    lanedata.loc[:,'weeke'] = 0
    lanedata.loc[:,'weekf'] = 0
    lanedata.loc[lanedata['t'].dt.weekday==0, 'weeka'] = 1 # Monday
    lanedata.loc[lanedata['t'].dt.weekday==1, 'weekb'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==2, 'weekc'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==3, 'weekd'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==4, 'weeke'] = 1
    lanedata.loc[lanedata['t'].dt.weekday==5, 'weekf'] = 1
    # time-of-day indicator
    lanedata['hour'] = lanedata['t'].dt.hour
    lanedata.loc[:,'toda'] = 0
    lanedata.loc[:,'todb'] = 0
    lanedata.loc[:,'todc'] = 0
    lanedata.loc[:,'todd'] = 0
    lanedata.loc[(0<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<6), 'toda'] = 1 # midnight to 6 am
    lanedata.loc[(7<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<9), 'todb'] = 1 # am peak
    lanedata.loc[(17<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<19), 'todc'] = 1 # pm peak
    lanedata.loc[(19<lanedata['t'].dt.hour) & (lanedata['t'].dt.hour<24), 'todd'] = 1 # night time
    
# ratio of flows in current lane to adjacent lanes
data1.loc[:,'fratio'] = data1['f']/data2['f']
# Use the average of the flow ratios with lane1 and lane3 as the ultimate flow ratio in lane2
# data2.loc[:,'fratio1'] = data2['f']/data1['f'] 
# data2.loc[:,'fratio3'] = data2['f']/data3['f']
data3.loc[:,'fratio'] = data3['f']/data2['f']

In [128]:
''' This block is used to determine the speed of lane2 is influenced by which lane (lan1 or lane3),
and the criterion is the flow ratio between lane 2 and lane1 and the flow ratio between lane 2 and lane3.
Bigger one will be dorminating!'''
data2.loc[:,'fratio1'] = data2['f']/data1['f']
data2.loc[:,'fratio3'] = data2['f']/data3['f']
data2.loc[:,'flag'] = 0
data2.loc[data2['fratio1']<data2['fratio3'], 'flag'] = 1 # using '<' recieves better result?
data2.loc[:,'fratio'] = 0
data2.loc[data2['flag']==1,'fratio'] = data2['fratio1']
data2.loc[data2['flag']==0,'fratio'] = data2['fratio3']
# consrtuting the instruments and endogeneity dataframe for lane2
data2_instru = data1[['t','o','f','ts','fi','s_do','s']]
data2_instru.loc[data2['flag']==0,['o','f','ts','fi','s_do','s']] = data3.loc[data2['flag']==0,['o','f','ts','fi','s_do','s']]

data2_instru.loc[:,'fratio'] = 0
data2_instru.loc[:,'fratio'] = data2.loc[:,'fratio']

In [129]:
# data2_instru.describe()

In [130]:
data1[['o','f','ts','fi','fratio','s_do','s']].describe()

,o,f,ts,fi,fratio,s_do,s
count,21910.000000,21910.000000,21910.0,21910.000000,21910.000000,21910.000000,21910.000000
mean,2.164340,27.810452,999.0,0.993930,0.572386,71.583136,69.133199
std,0.890855,16.440382,0.0,0.077677,0.181010,0.875484,1.076686
min,0.000000,1.000000,999.0,0.000000,0.090909,60.200000,62.600000
25%,1.300000,12.000000,999.0,1.000000,0.433333,71.400000,68.700000
50%,2.200000,28.000000,999.0,1.000000,0.619048,71.800000,69.200000
75%,2.900000,41.000000,999.0,1.000000,0.681818,71.900000,69.500000
max,7.300000,125.000000,999.0,1.000000,6.000000,75.000000,73.600000


In [131]:
# data1['fratio'] = data1['f']/data2['f']
# data1 = data1.dropna()
data1.dtypes

t          datetime64[ns]
f                   int64
o                 float64
s                 float64
s_do              float64
tf                  int64
tp                  int64
vht                 int64
vmt                 int64
s2                float64
lns               float64
ts                float64
fi                  int64
htfi                int64
tpia                int64
tpib                int64
seasona             int64
seasonb             int64
seasonc             int64
seasond           float64
weeka               int64
weekb               int64
weekc               int64
weekd               int64
weeke               int64
weekf               int64
hour                int64
toda                int64
todb                int64
todc                int64
todd                int64
fratio            float64
dtype: object

## 2 Prediction 

### 2.1 Error Examination

In [132]:
# data1[['f','fratio']] = data1[['f','fratio']].infer_objects()
# data2[['f','fratio']] = data2[['f','fratio']].infer_objects()
''' Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.
And the error come from other_var.
And after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'
are (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object 
to int and float, respectively.
'''

" Because when run the next code block, there is erro coming up: data must be numeric, string and categorical.\nAnd the error come from other_var.\nAnd after use data1[other_var].dtypes to visit the data type of all indexs in other_var, found the 'f' and 'fratio'\nare (object). So in order to solve this problem, use dataframe.infer_objects to change the data type from object \nto int and float, respectively.\n"

In [133]:
np.linalg.matrix_rank(data1[['o','f','ts','fi','fratio']])

5

In [134]:
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### 2.2 Predicting Used New Method (including downstream speed)

In [135]:
# 'exog' and 'endog' and 'instruments' cannot overlap to each other
error_result_new = pd.DataFrame(columns=['MAE1','MAE2','MAE3'])
speed_new1 = pd.DataFrame()
speed_new2 = pd.DataFrame()
speed_new3 = pd.DataFrame()
for month in range(12):
    data1_test = data1[data1['t'].dt.month == month+1]
    data1_train = data1[data1['t'].dt.month != month+1]
    data2_test = data2[data2['t'].dt.month == month+1]
    data2_train = data2[data2['t'].dt.month != month+1]
    data3_test = data3[data3['t'].dt.month == month+1]
    data3_train = data3[data3['t'].dt.month != month+1]
    
    data2_instru_test = data2_instru[data2_instru['t'].dt.month == month+1]
    data2_instru_train = data2_instru[data2_instru['t'].dt.month != month+1]
    
    other_var = ['seasona', 'seasonb', 'seasonc', 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd']
    instruments = ['o', 'f',  'ts', 'fi', 'fratio']
    data1_train.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_train.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_train.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
#     exo_var = instruments+other_var

    u1 = {'dependent': data1_train[['s1']],
         'exog': data1_train[other_var+['s_do1']],
         'endog': data1_train[['s2']],
         'instruments': data2_train[instruments+['s_do2']]}

    u2 = {'dependent': data2_train[['s2']],
         'exog': data2_train[other_var+['s_do2']],
         'endog': data2_instru_train[['s']],
         'instruments': data2_instru_train[instruments+['s_do']]}
    
    u3 = {'dependent': data3_train[['s3']],
         'exog': data3_train[other_var+['s_do3']],
         'endog': data3_train[['s2']],
         'instruments': data2_train[instruments+['s_do2']]}
    
    equations = dict(u1=u1, u2=u2, u3=u3)
    system_3sls = IV3SLS(equations)
    system_3sls_res = system_3sls.fit(method='gls', cov_type='unadjusted')
#     print(system_3sls_res)
    # system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations)
    # print(system_3sls_pre)
    
    data1_test.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_test.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_test.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    
    u1_test = {'dependent': data1_test[['s1']],
         'exog': data1_test[other_var+['s_do1']],
         'endog': data1_test[['s2']],
         'instruments': data2_test[instruments+['s_do2']]}

    u2_test = {'dependent': data2_test[['s2']],
         'exog': data2_test[other_var+['s_do2']],
         'endog': data2_instru_test[['s']],
         'instruments': data2_instru_test[instruments+['s_do']]}
    
    u3_test = {'dependent': data3_test[['s3']],
         'exog': data3_test[other_var+['s_do3']],
         'endog': data3_test[['s2']],
         'instruments': data2_test[instruments+['s_do2']]}
    
    equations_test = dict(u1=u1_test, u2=u2_test, u3=u3_test)
    system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations_test)
    # system_3sls_pre.describe()
    
    # from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    y_true1 = data1_test['s1']
    y_pred1 = system_3sls_pre['u1']
    y_true2 = data2_test['s2']
    y_pred2 = system_3sls_pre['u2']
    y_true3 = data3_test['s3']
    y_pred3 = system_3sls_pre['u3']
    # print(mse(y_true, y_pred))
#     print(mae(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))
    error_result_new.loc[month,'MAE1'] = mae(y_true1, y_pred1)
    error_result_new.loc[month,'MAE2'] = mae(y_true2, y_pred2)
    error_result_new.loc[month,'MAE3'] = mae(y_true3, y_pred3)
    speed_new1 = pd.concat([speed_new1,y_pred1])
    speed_new2 = pd.concat([speed_new2,y_pred2])
    speed_new3 = pd.concat([speed_new3,y_pred3])
#     speed_new1['new1'] = speed_new1['new1'].append(y_pred1) # this command results in a result of a month only.

E:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
E:\Anaconda3\lib\site-packages\linearmodels\utility.py:454: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


ValueError: Equation u1 regressor array is not full rank

### 2.3 Predicting Used Old Method (not including downstream speed)

In [ ]:
# 'exog' and 'endog' and 'instruments' cannot overlap to each other
error_result_old = pd.DataFrame(columns=['MAE1','MAE2','MAE3'])
speed_old1 = pd.DataFrame()
speed_old2 = pd.DataFrame()
speed_old3 = pd.DataFrame()
for month in range(12):
    data1_test = data1[data1['t'].dt.month == month+1]
    data1_train = data1[data1['t'].dt.month != month+1]
    data2_test = data2[data2['t'].dt.month == month+1]
    data2_train = data2[data2['t'].dt.month != month+1]
    data3_test = data3[data3['t'].dt.month == month+1]
    data3_train = data3[data3['t'].dt.month != month+1]
    
    data2_instru_test = data2_instru[data2_instru['t'].dt.month == month+1]
    data2_instru_train = data2_instru[data2_instru['t'].dt.month != month+1]
    
    other_var = ['seasona', 'seasonb', 'seasonc', 'weeka', 'weekb', 'weekc', 'weekd', 'weeke', 'weekf', 'toda', 'todb', 'todc', 'todd']
    instruments = ['o', 'f',  'ts', 'fi', 'fratio']
    data1_train.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_train.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_train.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
#     exo_var = instruments+other_var

    u1 = {'dependent': data1_train[['s1']],
         'exog': data1_train[other_var],
         'endog': data1_train[['s2']],
         'instruments': data2_train[instruments]}

    u2 = {'dependent': data2_train[['s2']],
         'exog': data2_train[other_var],
         'endog': data2_instru_train[['s']],
         'instruments': data2_instru_train[instruments]}
    
    u3 = {'dependent': data3_train[['s3']],
         'exog': data3_train[other_var],
         'endog': data3_train[['s2']],
         'instruments': data2_train[instruments]}
    
    equations = dict(u1=u1, u2=u2, u3=u3)
    system_3sls = IV3SLS(equations)
    system_3sls_res = system_3sls.fit(method='gls', cov_type='unadjusted')
#     print(system_3sls_res)
    # system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations)
    # print(system_3sls_pre)
    
    data1_test.rename(columns={'s':'s1', 's_do':'s_do1'}, inplace=True)
    data2_test.rename(columns={'s':'s2', 's_do':'s_do2'}, inplace=True)
    data3_test.rename(columns={'s':'s3', 's_do':'s_do3'}, inplace=True)
    
    u1_test = {'dependent': data1_test[['s1']],
         'exog': data1_test[other_var],
         'endog': data1_test[['s2']],
         'instruments': data2_test[instruments]}

    u2_test = {'dependent': data2_test[['s2']],
         'exog': data2_test[other_var],
         'endog': data2_instru_test[['s']],
         'instruments': data2_instru_test[instruments]}
    
    u3_test = {'dependent': data3_test[['s3']],
         'exog': data3_test[other_var],
         'endog': data3_test[['s2']],
         'instruments': data2_test[instruments]}
    
    equations_test = dict(u1=u1_test, u2=u2_test, u3=u3_test)
    system_3sls_pre = system_3sls.predict(params = system_3sls_res.params, equations = equations_test)
    # system_3sls_pre.describe()
    
    # from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae
    y_true1 = data1_test['s1']
    y_pred1 = system_3sls_pre['u1']
    y_true2 = data2_test['s2']
    y_pred2 = system_3sls_pre['u2']
    y_true3 = data3_test['s3']
    y_pred3 = system_3sls_pre['u3']
    # print(mse(y_true, y_pred))
#     print(mae(y_true, y_pred))
#     print(mean_absolute_percentage_error(y_true, y_pred))
    error_result_old.loc[month,'MAE1'] = mae(y_true1, y_pred1)
    error_result_old.loc[month,'MAE2'] = mae(y_true2, y_pred2)
    error_result_old.loc[month,'MAE3'] = mae(y_true3, y_pred3)
    speed_old1 = pd.concat([speed_old1,y_pred1])
    speed_old2 = pd.concat([speed_old2,y_pred2])
    speed_old3 = pd.concat([speed_old3,y_pred3])
#     speed_new1['new1'] = speed_new1['new1'].append(y_pred1) # this command results in a result of a month only.

### 2.4 Result Exporting

In [ ]:
# speed_new1.columns = ['new1']
# speed_new2.columns = ['new2']
# speed_new3.columns = ['new3']
# speed_old1.columns = ['old1']
# speed_old2.columns = ['old2']
# speed_old3.columns = ['old3']

In [ ]:
speed = speed_new1
speed['new2'] = speed_new2
speed['new3'] = speed_new3
speed['old1'] = speed_old1
speed['old2'] = speed_old2
speed['old3'] = speed_old3
speed['observed1'] = data1['s']
speed['observed2'] = data2['s']
speed['observed3'] = data3['s']
speed.rename(columns={0: 'new1'}, inplace=True)
speed.describe()

In [ ]:
error_result = pd.DataFrame()
error_result['MAE_new1'] = error_result_new['MAE1']
error_result['MAE_new2'] = error_result_new['MAE2']
error_result['MAE_new3'] = error_result_new['MAE3']
error_result['MAE_old1'] = error_result_old['MAE1']
error_result['MAE_old2'] = error_result_old['MAE2']
error_result['MAE_old3'] = error_result_old['MAE3']
error_result

In [ ]:
speed.to_excel('G:\\Modeling Endogeneity\\Results\\100m_3lanes_speedtt.xlsx')
error_result.to_excel('G:\\Modeling Endogeneity\\Results\\100m_3lanes_errortt.xlsx')
data_up[['t','f1','f2','f3']].to_excel('G:\\Modeling Endogeneity\\Results\\100m_3lanes_flowtt.xlsx')